In [0]:
import os
os.listdir("/nuevomodelo")

In [0]:
display(dbutils.fs.ls("/nuevomodelo/modelrnn_nuevo2"))

In [0]:
import tensorflow as tf
# Paso 1: Copiar el modelo desde el almacenamiento Azure Storage al Directorio LOCAL(.OS) -- Obligatorio Poner file:
dbutils.fs.cp("/mnt/datalakemlopsd4m/presentation/modelrnn_nuevo", "file:/nuevomodelo/modelrnn_nuevo3", recurse=True)

In [0]:
# Paso 2: Cargar el Modelo ML desde el Directorio LOCAL(.OS) -- No es necesario poner file:
model2 = tf.keras.models.load_model("/nuevomodelo/modelrnn_nuevo3")

In [0]:
#Paso 3: Hace la prediccion con el modelo cargado del Azure Storage
model2.predict([[12,45,229,4,129]])